# CIFAR10 with MLPs
Simple starter notebook to benchmark your own MLP with PyTorch on the CIFAR-10 dataset.

OBS.:

- The main code is basically done, so focus on training the models and searching for the best hyperparameters and architectures.
- You are not required to use this exact code or even the PyTorch library.
- It is recommended to use execution environments with GPU access (such as Google Colab), since larger models will take more time to train.
- Remember to document the history of your experiments and which results motivated the changes in subsequent experiments.

In [ ]:
#@title Libs

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics

from tqdm import tqdm

In [ ]:
# -----------------------------------------------------------------------
# 1) TRANSFORMS SIMPLIFICADOS PARA TREINO (Data Augmentation LEVE)
# -----------------------------------------------------------------------
transform_train = transforms.Compose([
    # 1.a) RandomCrop (mais rápido que rotações e jitter)
    transforms.RandomCrop(32, padding=4),

    # 1.b) RandomHorizontalFlip (barato e costuma dar bom ganho)
    transforms.RandomHorizontalFlip(p=0.5),

    # Removemos RandomRotation e ColorJitter para economizar tempo
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    ),
])

# -----------------------------------------------------------------------
# 2) TRANSFORM PARA TESTE (APENAS NORMALIZAÇÃO)
# -----------------------------------------------------------------------
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    ),
])

# -----------------------------------------------------------------------
# 3) CRIAÇÃO DOS DATASETS USANDO OS NOVOS TRANSFORMS
# -----------------------------------------------------------------------
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform_train
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform_test
)

100%|██████████| 170M/170M [00:12<00:00, 13.4MB/s]


# You can change/optimize this as you want
- Different optimizers, activation functions, etc
- Automatic hyperparameters optimization (Optuna)
- Regularization techniques
- Validation set to track metrics during epochs
- Transform input data
- ...

## Experimento 1: Variação no Número de Camadas e Neurônios

### Aumentar a profundidade e o número de neurônios (por exemplo, de 3 camadas ocultas (64→128→64) para 3 camadas ocultas maiores (128→256→128)) dá mais capacidade ao modelo.

#### Porém, pode aumentar o risco de overfitting e também o custo computacional (tempo de treinamento).

In [ ]:
# -----------------------------------------------
# Experimento 1: MLP maior -> 3072 → 128 → 256 → 128 → 10
# -----------------------------------------------
class MLP(nn.Module):
    def __init__(self, input_size, num_classes, activation_function):
        super(MLP, self).__init__()
        self.activation_function = activation_function

        self.fc_input   = nn.Linear(input_size, 128)
        self.fc_hidden1 = nn.Linear(128, 256)
        self.fc_hidden2 = nn.Linear(256, 128)
        self.fc_output  = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.activation_function(self.fc_input(x))
        x = self.activation_function(self.fc_hidden1(x))
        x = self.activation_function(self.fc_hidden2(x))
        x = self.fc_output(x)
        return x


In [ ]:
#@title Defining metrics helper

def get_scores(targets, predictions):
    return {
        "accuracy": metrics.accuracy_score(targets, predictions),
        "balanced_accuracy": metrics.balanced_accuracy_score(targets, predictions),
        "precision": metrics.precision_score(targets, predictions, average="weighted"),
        "recall": metrics.recall_score(targets, predictions, average="weighted"),
        "f1_score": metrics.f1_score(targets, predictions, average="weighted")
    }

In [ ]:
#@title Hyperparameters
input_size = 32*32*3 # 32x32 RGB images
num_classes = 10

learning_rate = 0.001
num_epochs = 100
batch_size = 16
activation_function = nn.ReLU()

loss_function = nn.CrossEntropyLoss()

In [ ]:
#@title Loaders

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
#@title Training loop

# Build the model
mlp = MLP(input_size=input_size, num_classes=num_classes, activation_function=activation_function)
mlp.cuda()

# Setting optimizer up
optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

# Early stopping setup
best_loss = float('inf')
patience = 5
patience_counter = 0

# Start training epochs loop
for epoch in tqdm(range(num_epochs)):
  epoch_loss = 0.0
  for i, (images, labels) in enumerate(train_loader):
    images = images.view(-1,32*32*3).cuda() # flattenning images
    labels = labels.cuda()

    # Forward pass
    optimizer.zero_grad()
    outputs = mlp(images)

    # Backward pass
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

    if (i+1) % 1000 == 0:
      tqdm.write(f' Epoch {epoch + 1}/{num_epochs}, Step {i+1}/{len(train_dataset) // batch_size}, Loss: {loss}')

  epoch_loss /= len(train_loader)
  tqdm.write(f'Epoch {epoch+1} average loss: {epoch_loss:.4f}')

  # Early stopping using loss value
  if epoch_loss < best_loss:
    best_loss = epoch_loss
    patience_counter = 0
  else:
    patience_counter += 1
    if patience_counter >= patience:
      tqdm.write("Early stopping triggered.")
      break

  0%|          | 0/100 [00:08<?, ?it/s]

 Epoch 1/100, Step 1000/3125, Loss: 1.8286871910095215


  0%|          | 0/100 [00:22<?, ?it/s]

 Epoch 1/100, Step 2000/3125, Loss: 1.6592594385147095


  0%|          | 0/100 [00:30<?, ?it/s]

...

 Epoch 69/100, Step 2000/3125, Loss: 1.8682687282562256


 68%|██████▊   | 68/100 [27:26<12:24, 23.28s/it]

 Epoch 69/100, Step 3000/3125, Loss: 1.19283926486969


 68%|██████▊   | 68/100 [27:27<12:55, 24.22s/it]

Epoch 69 average loss: 1.3018
Early stopping triggered.


In [ ]:
#@title Evaluate model (accuracy, precision, recall)
mlp.eval()
predictions = []
labels = []
for images, label in test_loader:
  images = images.view(-1,32*32*3).cuda()
  label = label.cuda()

  output = mlp(images)
  _, predicted = torch.max(output,1)

  predictions.extend(predicted.cpu().numpy())
  labels.extend(label.cpu().numpy())

scores = get_scores(labels, predictions)
print("Scores of your model\n", scores)

Scores of your model
 {'accuracy': 0.5209, 'balanced_accuracy': np.float64(0.5208999999999999), 'precision': 0.5188828775879197, 'recall': 0.5209, 'f1_score': 0.5131900806169937}


## Experimento com Funções de Ativação

In [ ]:
#@title Experimento 2 — Variação da Função de Ativação

# -------------------------------------------------------
# Hipóteses:
# - ReLU: baseline (já vimos ~47–52% de acurácia com lr=1e-3)
# - LeakyReLU: evitar neurônios mortos, possível ganho leve
# - Tanh: centraliza em zero, mas pode saturar, treino mais lento
# - Sigmoid: saturação em [0,1], risco maior de vanishing gradient
# -------------------------------------------------------

# 1) Escolha APENAS UMA destas linhas por vez:
# activation_function = nn.ReLU()            # 3a: baseline
activation_function = nn.LeakyReLU(0.1)     # 3b: LeakyReLU
# activation_function = nn.Tanh()           # 3c: Tanh
# activation_function = nn.Sigmoid()        # 3d: Sigmoid

# -----------------------------------------------
# 2) MLP maior -> 3072 → 128 → 256 → 128 → 10
# -----------------------------------------------
class MLP(nn.Module):
    def __init__(self, input_size, num_classes, activation_function):
        super(MLP, self).__init__()
        self.activation_function = activation_function

        # Em vez de 64, 128, 64, usamos 128, 256, 128
        self.fc_input   = nn.Linear(input_size, 128)
        self.fc_hidden1 = nn.Linear(128, 256)
        self.fc_hidden2 = nn.Linear(256, 128)
        self.fc_output  = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.activation_function(self.fc_input(x))
        x = self.activation_function(self.fc_hidden1(x))
        x = self.activation_function(self.fc_hidden2(x))
        x = self.fc_output(x)
        return x

# -------------------------------
# 3) Hiperparâmetros e DataLoader
# -------------------------------
input_size = 32 * 32 * 3
num_classes = 10

learning_rate = 1e-3
num_epochs = 100
batch_size = 16
loss_function = nn.CrossEntropyLoss()

mlp = MLP(input_size=input_size, num_classes=num_classes, activation_function=activation_function)
mlp.cuda()

optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate)

# -----------------
# 4) Loop de Treino
# -----------------
best_loss = float('inf')
patience = 5
patience_counter = 0

for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 32*32*3).cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        outputs = mlp(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        if (i + 1) % 1000 == 0:
            tqdm.write(
                f'Epoch {epoch+1}/{num_epochs}, '
                f'Step {i+1}/{len(train_loader)}, '
                f'Loss: {loss.item():.4f}'
            )

    epoch_loss /= len(train_loader)
    tqdm.write(f'Epoch {epoch+1} average loss: {epoch_loss:.4f}')

    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            tqdm.write("Early stopping triggered.")
            break

# ------------------
# 5) Avaliação Final
# ------------------
mlp.eval()
predictions, labels_list = [], []
for images, labels in test_loader:
    images = images.view(-1, 32*32*3).cuda()
    labels = labels.cuda()

    output = mlp(images)
    _, predicted = torch.max(output, 1)
    predictions.extend(predicted.cpu().numpy())
    labels_list.extend(labels.cpu().numpy())

scores = {
    "activation_function": activation_function.__class__.__name__,
    "accuracy": metrics.accuracy_score(labels_list, predictions),
    "balanced_accuracy": metrics.balanced_accuracy_score(labels_list, predictions),
    "precision": metrics.precision_score(labels_list, predictions, average="weighted"),
    "recall": metrics.recall_score(labels_list, predictions, average="weighted"),
    "f1_score": metrics.f1_score(labels_list, predictions, average="weighted")
}
print("Scores (activation = ", activation_function.__class__.__name__, ")\n", scores)


  0%|          | 0/100 [00:07<?, ?it/s]

Epoch 1/100, Step 1000/3125, Loss: 1.9424


  0%|          | 0/100 [00:14<?, ?it/s]

Epoch 1/100, Step 2000/3125, Loss: 1.8067


  0%|          | 0/100 [00:21<?, ?it/s]

...

Epoch 81/100, Step 2000/3125, Loss: 1.7915


 80%|████████  | 80/100 [31:20<07:41, 23.08s/it]

Epoch 81/100, Step 3000/3125, Loss: 1.2290


 80%|████████  | 80/100 [31:21<07:50, 23.52s/it]


Epoch 81 average loss: 1.2664
Early stopping triggered.
Scores (activation =  LeakyReLU )
 {'activation_function': 'LeakyReLU', 'accuracy': 0.5424, 'balanced_accuracy': np.float64(0.5424), 'precision': 0.5621743551949822, 'recall': 0.5424, 'f1_score': 0.5385572976917209}


## Experimento 3: Batch_size 64 e L2 regularization



In [ ]:
# -------------------------------------------------------
# Hipóteses:
# - ReLU: baseline (já vimos ~47–52% de acurácia com lr=1e-3)
# - LeakyReLU: evitar neurônios mortos, possível ganho leve
# - Tanh: centraliza em zero, mas pode saturar, treino mais lento
# - Sigmoid: saturação em [0,1], risco maior de vanishing gradient
# -------------------------------------------------------

# 1) Escolha APENAS UMA destas linhas por vez:
# activation_function = nn.ReLU()            # 3a: baseline
activation_function = nn.LeakyReLU(0.1)     # 3b: LeakyReLU
# activation_function = nn.Tanh()           # 3c: Tanh
# activation_function = nn.Sigmoid()        # 3d: Sigmoid

# -----------------------------------------------
# 2) MLP maior -> 3072 → 128 → 256 → 128 → 10
# -----------------------------------------------
class MLP(nn.Module):
    def __init__(self, input_size, num_classes, activation_function):
        super(MLP, self).__init__()
        self.activation_function = activation_function

        self.fc_input   = nn.Linear(input_size, 128)
        self.fc_hidden1 = nn.Linear(128, 256)
        self.fc_hidden2 = nn.Linear(256, 128)
        self.fc_output  = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.activation_function(self.fc_input(x))
        x = self.activation_function(self.fc_hidden1(x))
        x = self.activation_function(self.fc_hidden2(x))
        x = self.fc_output(x)
        return x

# ------------------
# 3) Hiperparâmetros
# ------------------
input_size = 32 * 32 * 3
num_classes = 10

learning_rate = 1e-3
num_epochs = 100
batch_size = 64
loss_function = nn.CrossEntropyLoss()

mlp = MLP(input_size=input_size, num_classes=num_classes, activation_function=activation_function)
mlp.cuda()

optimizer = torch.optim.Adam(
  mlp.parameters(),
  lr=learning_rate,
  weight_decay=1e-5      # adiciona L2 regularization
)

# -----------------
# 4) Loop de Treino
# -----------------
best_loss = float('inf')
patience = 5
patience_counter = 0

for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 32*32*3).cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        outputs = mlp(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        if (i + 1) % 1000 == 0:
            tqdm.write(
                f'Epoch {epoch+1}/{num_epochs}, '
                f'Step {i+1}/{len(train_loader)}, '
                f'Loss: {loss.item():.4f}'
            )

    epoch_loss /= len(train_loader)
    tqdm.write(f'Epoch {epoch+1} average loss: {epoch_loss:.4f}')

    # Early stopping
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            tqdm.write("Early stopping triggered.")
            break

# ------------------
# 5) Avaliação Final
# ------------------
mlp.eval()
predictions, labels_list = [], []
for images, labels in test_loader:
    images = images.view(-1, 32*32*3).cuda()
    labels = labels.cuda()

    output = mlp(images)
    _, predicted = torch.max(output, 1)
    predictions.extend(predicted.cpu().numpy())
    labels_list.extend(labels.cpu().numpy())

scores = {
    "activation_function": activation_function.__class__.__name__,
    "accuracy": metrics.accuracy_score(labels_list, predictions),
    "balanced_accuracy": metrics.balanced_accuracy_score(labels_list, predictions),
    "precision": metrics.precision_score(labels_list, predictions, average="weighted"),
    "recall": metrics.recall_score(labels_list, predictions, average="weighted"),
    "f1_score": metrics.f1_score(labels_list, predictions, average="weighted")
}
print("Scores (activation = ", activation_function.__class__.__name__, ")\n", scores)

  1%|          | 1/100 [00:21<34:48, 21.10s/it]

Epoch 1 average loss: 1.8168


  2%|▏         | 2/100 [00:40<32:49, 20.10s/it]

Epoch 2 average loss: 1.6553


  3%|▎         | 3/100 [01:00<32:41, 20.23s/it]

...

Epoch 98 average loss: 1.1591


 99%|█████████▉| 99/100 [32:42<00:19, 19.61s/it]

Epoch 99 average loss: 1.1558


100%|██████████| 100/100 [33:01<00:00, 19.81s/it]


Epoch 100 average loss: 1.1579
Scores (activation =  LeakyReLU )
 {'activation_function': 'LeakyReLU', 'accuracy': 0.5475, 'balanced_accuracy': np.float64(0.5475000000000001), 'precision': 0.5792963719302255, 'recall': 0.5475, 'f1_score': 0.5484232659070362}


## Experimento 4: SGD com momentum

In [ ]:
def get_scores(targets, predictions):
    return {
        "accuracy": metrics.accuracy_score(targets, predictions),
        "balanced_accuracy": metrics.balanced_accuracy_score(targets, predictions),
        "precision": metrics.precision_score(targets, predictions, average="weighted"),
        "recall": metrics.recall_score(targets, predictions, average="weighted"),
        "f1_score": metrics.f1_score(targets, predictions, average="weighted")
    }

def evaluate_model(model, data_loader):
    """
    Avalia model em data_loader e retorna dict com
    accuracy, balanced_accuracy, precision, recall e f1_score.
    """
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.view(-1, 32*32*3).cuda()
            labels = labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return get_scores(all_labels, all_preds)

In [ ]:
# -------------------------------------------------------
# Hipóteses:
# - ReLU: baseline (já vimos ~47–52% de acurácia com lr=1e-3)
# - LeakyReLU: evitar neurônios mortos, possível ganho leve
# - Tanh: centraliza em zero, mas pode saturar, treino mais lento
# - Sigmoid: saturação em [0,1], risco maior de vanishing gradient
# -------------------------------------------------------

# 1) Escolha APENAS UMA destas linhas por vez:
# activation_function = nn.ReLU()            # 3a: baseline
activation_function = nn.LeakyReLU(0.1)     # 3b: LeakyReLU
# activation_function = nn.Tanh()           # 3c: Tanh
# activation_function = nn.Sigmoid()        # 3d: Sigmoid

# -----------------------------------------------
# 2) MLP maior -> 3072 → 128 → 256 → 128 → 10
# -----------------------------------------------
class MLP(nn.Module):
    def __init__(self, input_size, num_classes, activation_function):
        super(MLP, self).__init__()
        self.activation_function = activation_function

        # Em vez de 64, 128, 64, usamos 128, 256, 128
        self.fc_input   = nn.Linear(input_size, 128)
        self.fc_hidden1 = nn.Linear(128, 256)
        self.fc_hidden2 = nn.Linear(256, 128)
        self.fc_output  = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.activation_function(self.fc_input(x))
        x = self.activation_function(self.fc_hidden1(x))
        x = self.activation_function(self.fc_hidden2(x))
        x = self.fc_output(x)
        return x

# ------------------
# 3) Hiperparâmetros
# ------------------
input_size = 32 * 32 * 3
num_classes = 10

learning_rate = 1e-3
num_epochs = 100
batch_size = 64
loss_function = nn.CrossEntropyLoss()

mlp = MLP(input_size=input_size, num_classes=num_classes, activation_function=activation_function)
mlp.cuda()

optimizer = torch.optim.SGD(
  mlp.parameters(),
  lr=learning_rate,
  momentum=0.9
)

# -----------------
# 4) Loop de Treino
# -----------------
best_loss = float('inf')
patience = 5
patience_counter = 0

for epoch in tqdm(range(1, num_epochs + 1)):
    # ---- Modo Treino ----
    mlp.train()
    epoch_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 32*32*3).cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        outputs = mlp(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        if (i + 1) % 1000 == 0:
            tqdm.write(
                f'Epoch {epoch}/{num_epochs}, '
                f'Step {i+1}/{len(train_loader)}, '
                f'Loss (batch): {loss.item():.4f}'
            )

    epoch_loss /= len(train_loader)
    tqdm.write(f'[Epoch {epoch}/{num_epochs}] Loss médio do treino: {epoch_loss:.4f}')

    # ---- Avaliação no Conjunto de Treino e Teste ----
    train_scores = evaluate_model(mlp, train_loader)
    test_scores = evaluate_model(mlp, test_loader)

    tqdm.write(
        f"Epoch {epoch} — Train 📈 "
        f"acc: {train_scores['accuracy']:.4f}, "
        f"prec: {train_scores['precision']:.4f}, "
        f"rec: {train_scores['recall']:.4f}, "
        f"f1: {train_scores['f1_score']:.4f}"
    )
    tqdm.write(
        f"Epoch {epoch} — Test  🔍 "
        f"acc: {test_scores['accuracy']:.4f}, "
        f"prec: {test_scores['precision']:.4f}, "
        f"rec: {test_scores['recall']:.4f}, "
        f"f1: {test_scores['f1_score']:.4f}"
    )

    # ---- Early Stopping com base no loss de treino ----
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            tqdm.write("Early stopping acionado.")
            break


  0%|          | 0/100 [00:19<?, ?it/s]

[Epoch 1/100] Loss médio do treino: 2.2048


  1%|          | 1/100 [00:40<1:06:15, 40.15s/it]

Epoch 1 — Train 📈 acc: 0.2484, prec: 0.2609, rec: 0.2484, f1: 0.1991
Epoch 1 — Test  🔍 acc: 0.2650, prec: 0.2599, rec: 0.2650, f1: 0.2160


  1%|          | 1/100 [00:58<1:06:15, 40.15s/it]

...

Epoch 99 — Train 📈 acc: 0.5684, prec: 0.5726, rec: 0.5684, f1: 0.5680
Epoch 99 — Test  🔍 acc: 0.5597, prec: 0.5597, rec: 0.5597, f1: 0.5555


 99%|█████████▉| 99/100 [1:06:14<00:40, 40.11s/it]

[Epoch 100/100] Loss médio do treino: 1.1981


100%|██████████| 100/100 [1:06:36<00:00, 39.96s/it]

Epoch 100 — Train 📈 acc: 0.5769, prec: 0.5748, rec: 0.5769, f1: 0.5710
Epoch 100 — Test  🔍 acc: 0.5523, prec: 0.5513, rec: 0.5523, f1: 0.5425
